
## In this notebook, you will see all the steps sequentially performed to be able to utilize the complete functionality of OAB framework. The steps are as follows :
0. SETUP
1. DATA
2. DATA SELECTION
3. PREPROCESSING
4. SAMPLING
5. ALGORITHM TRAINING AND TESTING
6. EVALUATION
7. SHOW BENCHMARK RESULTS
8. REPRODUCIBILTY
9. EXTENDING THE BENCHMARK(with own Algorithm)

This notebook focuses on <b>Semisupervised Image Data</b>. Let's begin!

# **0. SETUP**

`oab` framework can be integrated in your Python environment  as a PyPi package  using the following command:

In [ ]:
#ID 1(0)

%%capture
# pip install oab
!pip install example-pkg-jd-kiel --extra-index-url=https://test.pypi.org/simple/

## **0.1 NOTEBOOK AND CELL STRUCTURE** 

In this notebook there are certain sections where the user s required to enter its own information which are marked as comments of the form :

<b>### ADD YOUR CODE ###</b>  , so <b>###</b> can be searched to know what are those sections.

All cells are assigned an ID, as a comment at the top of the cell,for example as: <b>#ID 10(5)</b>, where 10 denotes the cell ID and 5 denotes the Section.

# **1. DATA**

First of all, we will have a look at the Datasets that are pre-installed in OAB which can be used for benchmarking

In [1]:
#ID 2(1)

#importing the necessary functions and internal variables 

import sys
import os
from os import listdir
#from os.path import isfile, join
from datetime import datetime 
from pathlib import Path
sys.path.append('../..')             #setting the current directory path
%load_ext autoreload
%autoreload 2


#other  necessary imports
from oab.data.classification_dataset import ClassificationDataset
from oab.data.semisupervised import SemisupervisedAnomalyDataset
from oab.data.abstract_classes import AnomalyDataset
from oab.algorithms.abstract_classes import AbstractWrapperToRecipe,AbstractWrapperFromRecipe
import yaml
from ruamel.yaml import YAML,RoundTripRepresenter
import numpy as np
from oab.data.utils_image import mvtec_ad_suffixes, mvtec_ad_datasets, mvtec_ad_color_datasets, image_datasets, url_dict, reshape_dict


lst_all_datasetnames =image_datasets
for i,dataset in enumerate(lst_all_datasetnames):
    print(f"{i}.{dataset}")

0.mnist
1.fashion_mnist
2.cifar10
3.cifar100
4.mvtec_ad_carpet
5.mvtec_ad_grid
6.mvtec_ad_leather
7.mvtec_ad_tile
8.mvtec_ad_wood
9.mvtec_ad_bottle
10.mvtec_ad_cable
11.mvtec_ad_capsule
12.mvtec_ad_hazelnut
13.mvtec_ad_metal_nut
14.mvtec_ad_pill
15.mvtec_ad_screw
16.mvtec_ad_toothbrush
17.mvtec_ad_transistor
18.mvtec_ad_zipper
19.crack



`oab` provides a variety of image datasets that can easily be loaded, in either of the following ways: 

`1.` If a user is interested in using her own image dataset  loading **via local folder directory**, the following steps have to be followed: (1) Ensure that the format is readable by oab. This requires there to be a folder for the dataset with the subfolders `normal` and `anomaly`. Naturally, all normal images are in the folder `normal` and all images of anomalies in the folder `anomaly`. (2) Based on this folder structure, the dataset can be loaded.


"**Local folder structure - without URL usage**" :
```
dataset_name
        │
        ├── normal
        │    
        └── anomaly
``` 

`2.` If user's dataset is provided **via a URL**, then it would be downloaded and stored in the OAB's "datasets" folder, given that it is already formatted as per the required folder heirarchy, where the folder :

`good`(which should not be renamed) : contains normal images whereas 

`anomaly_folder_n`(can be renamed):  contains anomalous images



"**Uploaded folder structure - with URL usage**" :
```
dataset_name
        │
        ├── train
        │   ├── good
        │
        │  
        └── test
            ├── good
            ├── anomaly_folder_1
            ├── ...
            ├── ...
            ├── anomaly_folder_2
```
 
Note: Alternatively, it is of course possible to load the images into `numpy` arrays and treat them as if they were tabular data. If this approach is to be followed, please look at the notebooks for tabular data.


- Notes: Limited to 256x256

Below,we define a helper function for loading dataset.

In [ ]:
#ID 3(1)

def add_own_dataset(dataset_name: str, bw: bool =False):
  """
  Helper that adds a dataset name to some internal variables used by oab.
  This is a workaround as the functionality to load an own dataset where files 
  are stored in a directory was not yet available when the paper was submitted.

  :param dataset_name: Name of the dataset
  :param bw: If the dataset is black-and-white

  :returns: the name which is to be used when loading the dataset.
  """
  name_mvtec_prefix = "mvtec_ad_" + dataset_name
  image_datasets.append(name_mvtec_prefix)
  mvtec_ad_suffixes.append(dataset_name)
  mvtec_ad_datasets.append(name_mvtec_prefix)
  if bw:
    mvtec_ad_bw_datasets.append(name_mvtec_prefix)
    reshape_dict[name_mvtec_prefix]=(256,256)
  else:
    mvtec_ad_color_datasets.append(name_mvtec_prefix)
    reshape_dict[name_mvtec_prefix]=(256,256,3)
  # url is just used for downloading the dataset if not already available offline and  for reproducibility
  # url_dict['myImageDataset']= 'ftp://<myftpserverurl>/myImageDataset.tar.xz 

  return name_mvtec_prefix 



  

In [ ]:
#ID 4(1)
#timestamp is set  for this run  

time=datetime.now().strftime("%Y%m%d%H%M%S")

### ADD YOUR OWN BENCHMARK NAME ###
benchmark_name="Paper_A"

#creating folder for this benchmark with current timestamp name to store recipes of this run
if not os.path.exists(f"{os.getcwd()}/{benchmark_name}/{time}"): #creating directory for this benchhmark run for storing recipes
    os.makedirs(f"{os.getcwd()}/{benchmark_name}/{time}")

In [ ]:
#ID 5(1)


#### ADD YOUR DATASETNAME(S)  HERE ###

own_datasets_list=["myImageDataset"]  # More of user's own datasets can be added in this list
oab_datasets_list=['mnist']   # More of OAB's datasets can be added to this list


# 'myImageDataset' is the name of the Dataset(which the user loads for benchmarking as well as the name of the folder containing normal and anomaly folders
#  which further contain the respective images
#  and make sure folder structure is correct ({dataset_folder}/{name-without mvtec_ad_}/[normal/anomaly])

dataset_folder='datasets'     #all image datasets folders are stored in this directory
 
new_recipe_path=f"{benchmark_name}/{time}/{time}_{benchmark_name}" # file path for new recipe created in this run    
    
# calling the helper function to update internal OAB variables 
mvtec_ad_own_datasets_list=[]
for dataset_name in own_datasets_list:
 mvtec_ad_own_datasets_list.append(add_own_dataset(dataset_name))
print( f" Dataset(s) successfully added as :  {mvtec_ad_own_datasets_list}")




#Now,  we'll have a look at all the datasets again which are pre-installed in OAB,after adding own_datasets 
lst_all_datasetnames =image_datasets
print("Datasets of OAB:")
for i,dataset in enumerate(lst_all_datasetnames):
    print(f"{i}.{dataset}")

 Dataset(s) successfully added as :  ['mvtec_ad_myImageDataset']
Datasets of OAB:
0.mnist
1.fashion_mnist
2.cifar10
3.cifar100
4.mvtec_ad_carpet
5.mvtec_ad_grid
6.mvtec_ad_leather
7.mvtec_ad_tile
8.mvtec_ad_wood
9.mvtec_ad_bottle
10.mvtec_ad_cable
11.mvtec_ad_capsule
12.mvtec_ad_hazelnut
13.mvtec_ad_metal_nut
14.mvtec_ad_pill
15.mvtec_ad_screw
16.mvtec_ad_toothbrush
17.mvtec_ad_transistor
18.mvtec_ad_zipper
19.crack
20.mvtec_ad_myImageDataset


Now, If this dataset(s) was already stored in the dataset_folder, structured as mentioned in the initial description of this section above, then we have to create the file "applied_modification.txt" in Path(dataset_folder)/dataset_name/ "applied_modification.txt"). If this is not available in the location, then we download the dataset from the given URL(When data is downloaded for the URL, the orientation of the folders as well as the image resizing operation is performed and  information about that is stored in "applied_modification.txt")

In [ ]:
#ID 6(1)
for dataset_name in own_datasets_list:
  open(Path(dataset_folder)/dataset_name/"applied_modification.txt", "w") 
  
  

# **2. DATA SELECTION**


Datasets can either be loaded directly as anomaly datasets or as classification datasets. In the former case, the dataset is automatically fully prepared and ready for sampling. In the latter case, further preprocessing is still possible and necessary.

Note that the automatic preprocessing for image datasets is to scale each value by `1/255`.

**After adding own dataset(s) in #ID 5 and #ID 6,the user is able to load own dataset(s) using this method :**

In [ ]:
#ID 7(2)

from oab.data.load_image_dataset import _load_image_dataset
datasets={}  # contain dataset objects of own dataset names 


for dataset_name in mvtec_ad_own_datasets_list:  # loading own datasets 
    datasets[dataset_name[9:]]=_load_image_dataset(dataset_name ,anomaly_dataset=False,semisupervised=True)
    

### **2.1 Load anomaly detection datasets (with or without further preprocessing)**

In this section, we load some pre-installed data sets, namely the transistor dataset from MVTecAD , Cifar10 as well as MNIST datasets. This can be achieved using the `load_dataset` function. By default, it creates an anomaly dataset from which sampling is directly possible  and first create classsifcation dataset and then anomaly dataset,either with the preprocessing applied (`preprocess_classification_dataset=True`) i.e. Scaling is performed on all values by the factor of 1/255, or without (`preprocess_classification_dataset=False`, default).

`In our case` we set have already imported own datasets with `anomaly_dataset=False ` and `preprocess_classification_dataset=False` in <b>#ID 7(2)</b> and we will also load the OAB datasets in the same way in <b>#ID 9(2)</b>

If this is to be used for semisupervised anomaly detection, `semisupervised=True` needs to be specified.

Note that as discussed in the paper, multiclass classification datasets like Cifar10 and MNIST are loaded with the class label `0` as normal label and all other labels as anomaly labels by default. (Alternatively, `oab` can automatically iterate through all classes as normal classes. This is not covered here.)

In [ ]:
#ID 8(2)

# imports
from oab.data.load_dataset import load_dataset

In [ ]:
#ID 9(2)

#### ADD YOUR OWN NUMBER OF DATASETS AND FROM OAB FOR BENCHMARKING  ###


for dataset_name in oab_datasets_list:  # loading benchmark's datasets
    datasets[dataset_name]=load_dataset(dataset_name,anomaly_dataset=False,preprocess_classification_dataset=False, semisupervised=True)
print(datasets)

{'myImageDataset': <oab.data.classification_dataset.ClassificationDataset object at 0x7fb6f83e0430>, 'mnist': <oab.data.classification_dataset.ClassificationDataset object at 0x7fb6f83e0c10>}


# **3. PREPROCESSING**

The  resizing of images(only when dataset is downloaded using URL) and scaling of images has already been performed while loading the datasets as shown in previous the Sections.

Standard preprocessing steps like deleting columns, encoding categorical values differently, or removing missing values do not apply to image data. Therefore, these methods (as well as own preprocessing steps and how these are captured) are covered in the tabular dataset benchmarks.

Here, we only show two preprocessing steps that are applied to datasets in `preprocess_datasets`(loaded in 2.2), which can also be performed individually depending upon requirement :
- `Scale` all values by `1/255`.
- `Transform the dataset into an anomaly dataset` for semisupervised anomaly detection by setting the class label `0` to normal and all other class labels to anomalous.

Firstly, we will define  helper functions which stores dataset information in a recipe file

In [ ]:
#ID 11 (3)                            SCALING APPLIED

#used imports from #ID 3                                          
for dataset_name in datasets:
    
    datasets[dataset_name].scale(scaling_factor=1/255)
    operations=datasets[dataset_name].operations_performed
    #NEW RECIPE CREATED
    datasets[dataset_name].write_operations_to_yaml(f"{new_recipe_path}_{dataset_name}_preprocessing.yaml")


Scaling performed on datasets! 


The individual recipe files now contains information about how to preprocess(i.e. perform scaling) the datasets

In [ ]:
#ID 12(3)

for (i,dataset_name) in enumerate(datasets):
    print(f"{i+1}.{new_recipe_path}_{dataset_name}_preprocessing.yaml\n")
    !cat {new_recipe_path}_{dataset_name}_preprocessing.yaml

1.Paper_A/20211207031059/20211207031059_Paper_A_myImageDataset_preprocessing.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098
2.Paper_A/20211207031059/20211207031059_Paper_A_mnist_preprocessing.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098


In [ ]:
#ID 13(3)                            ANOMALY-DATASET CONVERSION PERFORMED

#used import from #ID 2 


#recipe_path=f"{benchmark_name}/{time}_{benchmark_name}_recipe.yaml"                                           

datasets_ad={}    
    # for storing dataset objects converted to anomaly-dataset
for dataset_name in datasets:   
     datasets_ad[dataset_name]= SemisupervisedAnomalyDataset(classification_dataset=datasets[dataset_name],
                                                       normal_labels=0,
                                                       yamlpath_append=f"{new_recipe_path}_{dataset_name}_preprocessing.yaml")   
     
        
                                                                            
print("datasets after adding anomaly-conversion datasets: ")    
print(datasets_ad)

datasets after adding anomaly-conversion datasets: 
{'myImageDataset': <oab.data.semisupervised.SemisupervisedAnomalyDataset object at 0x7fb63666b2e0>, 'mnist': <oab.data.semisupervised.SemisupervisedAnomalyDataset object at 0x7fb6366459a0>}


In [ ]:
#ID 14(3)

for (i,dataset_name) in enumerate(datasets):
    print(f"{i+1}.{new_recipe_path}_{dataset_name}_preprocessing.yaml\n")
    !cat {new_recipe_path}_{dataset_name}_preprocessing.yaml

1.Paper_A/20211207031059/20211207031059_Paper_A_myImageDataset_preprocessing.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098
anomaly_dataset:
  arguments:
    normal_labels: 0
    anomaly_labels:
2.Paper_A/20211207031059/20211207031059_Paper_A_mnist_preprocessing.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098
anomaly_dataset:
  arguments:
    normal_labels: 0
    anomaly_labels:


# **4. SAMPLING**

Here, we define the sampling parameters to sample from the datasets

In [ ]:
#ID 15(4)

### ADD YOUR OWN SAMPLING PARAMETERS ###

# sampling parameters
training_split = 0.7                 # Specifies the proportion of normal data points that will be used during training
max_contamination_rate = 0.5         # Maximum contamination rate of the test set. If this is exceeded, not all anomalies that exist are sampled
n_steps = 1        # n_steps=10      # Number of samples to be taken

#These below are the possible sampling types to sample from datasets
sampling_types=['semisupervised_multiple','semisupervised_explicit_numbers_single','semisupervised_training_split_multiple','semisupervised_training_split_single']


sampling_type='semisupervised_training_split_multiple'  #by default for this run

sampling_params={'training_split':training_split,'max_contamination_rate':max_contamination_rate,'n_steps':n_steps,'flatten_images':False} 


The above sampling parameters are utilized in
<b>#ID 27(5) </b> 
for sampling the datasets(except for the pre-installed mv_tec_ad_datasets) before training the algorithms.

In [ ]:
#ID 16(4)

# Adding own and oab's {dataset_name:dataset_object} pair to benchmarking_datasets 
benchmarking_datasets={}

for (x,y) in datasets_ad.items():
    benchmarking_datasets[x]=y

print(benchmarking_datasets)    

{'myImageDataset': <oab.data.semisupervised.SemisupervisedAnomalyDataset object at 0x7fb63666b2e0>, 'mnist': <oab.data.semisupervised.SemisupervisedAnomalyDataset object at 0x7fb6366459a0>}


This dictionary <b>benchmarking_datasets</b> will be used for the Benchmarking as it contains all the information:
    
    
    1.dataset_name
    2.final_dataset_object(preprocessed and anomaly-converted)
 



# **5. ALGORITHM TRAINING AND TESTING**

We first download and import algorithms used for anomaly decection.

In [ ]:
#ID 17(5)

import wget

wget.download('https://raw.githubusercontent.com/ISDM-CAU-Kiel/oab/master/notebooks/benchmark_image/cae_ocsvm.py', "cae_ocsvm.py")
wget.download('https://raw.githubusercontent.com/ISDM-CAU-Kiel/oab/master/notebooks/benchmark_image/cae_iforest.py', "cae_iforest.py")
wget.download('https://raw.githubusercontent.com/ISDM-CAU-Kiel/oab/master/notebooks/benchmark_image/conv_ae.py', "conv_ae.py")
### ADD your algo import here ###

In [ ]:
#ID 18(5)

# used imports from #ID 2
# used imports from #ID 8


## ADD your algo(s) import here ###
#from module_name import class_name

from conv_ae import ConvAutoEncoder 
from cae_ocsvm import CAEOCSVM
from cae_iforest import CAEIForest
from oab.evaluation import EvaluationObject, ComparisonObject,all_metrics
import tensorflow as tf
import random
import torch
import numpy as np



Firstly, we define hyperparameters for all algorithms and choose for benchmarking:

In [ ]:
#ID 19(5)

   
### Extend Algos dictionary with own algorithm specifications as shown below for OAB algorithms ###   

### ADD YOUR OWN (HYPER)PARAMETERS AND THEIR VALUES FOR PRE-INSTALLED ALGOS###


lst_benchmark_algorithms = [
    
    {
       "algo_module_name": "cae_iforest",   
       "algo_class_name": "CAEIForest",
       "algo_name_in_result_table": "CAE v2",
       "algo_parameters": {"CAE_parameters": {'latent_dim': 100, 'epochs': 50, 'verbose': 0}, "IForest_parameters": {'random_state': 42} },
        "fit": {'method_name': 'fit', 'params': {}}, 
        "decision_function": {'method_name': 'decision_function', 'params': {}}
        }]
'''    
      ### ConvAutoEncoder is not used, uncomment to use for benchmarking
      {
       "algo_module_name": "conv_ae"   ,
       "algo_class_name": "ConvAutoEncoder",
       "algo_name_in_result_table": "CAE v1",
       "algo_parameters":   {'latent_dim': 100, 'epochs': 50, 'verbose': 0},
        "fit": {'method_name': 'fit', 'params': {}}, 
       "decision_function": {'method_name': 'decision_function', 'params': {}}
       }  ]
 
 ### CAEOIforest is not used, uncomment ###    
       ]



### CAEOCSVM is not used, uncomment ### 
    {   
       "algo_module_name": "cae_ocsvm" , 
       "algo_class_name": "CAEOCSVM",
       "algo_name_in_result_table": "CAE v3",
       "algo_parameters": {"CAE_parameters": {'latent_dim': 100, 'epochs': 50, 'verbose': 0},"OCSVM_parameters": {'degree': 3}},
       "fit": {'method_name': 'fit', 'params': {}}, 
       "decision_function": {'method_name': 'decision_function', 'params': {}}
    } 
]    
'''      
  

own_algorithms=[]  #add your own algorithm dictionaries like above for pre-installed algos

lst_benchmark_algorithms.extend(own_algorithms)


#seed value defined for ths benchmark run for obtaining consistent results 
seed=42 # seeting seed value for consistency in reproducibility of benchmark   

<b>LOAD YOUR RECIPES</b> to be repdroduced and use it in the current benchmark run, lets,see  <b>how it looks</b>!

In [ ]:
#ID 21(5)
recipe_files_algos={'conv_ae': "Paper_B/202120211206212223_Paper_B_conv_ae[ConvAutoEncoder]_recipe.yaml"} #add recipe files to this dictionary

for (i,file) in enumerate(recipe_files_algos):
    print(f"{i+1}.{file}\n")
    !cat {recipe_files_algos[file]} 

1.conv_ae

decision_function:
  method_name: decision_function
  params: {}
fit:
  method_name: fit
  params: {}
init:
  params:
    epochs: 50
    latent_dim: 100
    verbose: 0
seed: 42    


In [ ]:
# ID 20(5)
recipe_files_datasets={} #add recipe files to this dictionary

for (i,file) in enumerate(recipe_files_datasets):
    print(f"{i+1}.{file}\n")
    !cat {recipe_files_datasets[file]} 
   

In [ ]:
#ID 22(5)
#Loading datasets from recipes wth preprocessing and anomaly-conversion applied

for dataset_name in recipe_files_datasets:  # loading benchmark's datasets
    cd=load_dataset(dataset_name,anomaly_dataset=False,preprocess_classification_dataset=False, semisupervised=True)
    cd.perform_operations_from_yaml(filepath=recipe_files_datasets[dataset_name]) #performing preprocess operations
    ad=cd.tranform_from_yaml(filepath=recipe_files_datasets[dataset_name],semisupervised=True) #anomaly-dataset conversion
    benchmarking_datasets[dataset_name]=ad   # dding datasets from recipe to benchmarking_datasets
    
    #creating new preprocessing recipe from Paper_B to  Paper_A's(i.e. who runs this benchhmark ) folder
    yaml=YAML(typ='rt')
    yaml_content = yaml.load(Path("./") / recipe_files_datasets[dataset_name])
    yaml.dump(yaml_content,Path("./")/f"{new_recipe_path}_{dataset_name}_preprocessing.yaml")
  


In [ ]:
#ID 23(5)
#Loading Algorithms from recipes and using it for benchmarking

import re
from json import loads,dumps
               
for algo_name in recipe_files_algos:
    yaml=YAML(typ='rt')
    yaml_content = yaml.load(Path("./") / recipe_files_algos[algo_name])
    class_name=re.findall(r'\[.*?\]',recipe_files_algos[algo_name]) # get class_name from filename of algo   
    recipe_algo={   
       "algo_module_name": algo_name , 
       "algo_class_name": class_name[0][1:-1],
       "algo_name_in_result_table": algo_name ,
       "algo_parameters":loads(dumps(yaml_content['init']['params'])),
       "fit":loads(dumps(yaml_content['fit'])), 
       "decision_function": loads(dumps(yaml_content['decision_function'])),
    }
    seed=loads(dumps(yaml_content['seed']))          # setting seed value from recipe
    if recipe_algo not in lst_benchmark_algorithms:   
      lst_benchmark_algorithms.append(recipe_algo)
      # append recipe algo in benchmarking algos for this run
    


Now, For every benchmark dataset , we sample from that dataset to train the algorithms and then predict the outcomes for each dataset with each algortihm and then store results in a evaluation object, which is then added to the comparison object to show the final Benchmarking results

For benchmark run, there are a couple of minor <b>bug fixes</b>  found in the OAB modules, which are dealt with below:

In [ ]:
#ID 24(5)

#               1.-------------------------------
# import manually (and due to a missing import, assign into correct module)
from oab.data.utils import _make_yaml,_append_to_yaml,_append_sampling_to_yaml
from oab.data import semisupervised
semisupervised._append_sampling_to_yaml = _append_sampling_to_yaml


#               2.--------------------------------
#from oab.algorithms.semisupervised_wrapper import SemisupervisedWrapperToRecipe (syntactical error in class)
from typing import Dict
class SemisupervisedWrapperToRecipe(AbstractWrapperToRecipe):

    def __init__(self):
        pass

    def track_init(self, obj, params: Dict = {}):
        self.init_dict = {'params': params}
        return obj(**params)

    def track_fit(self, x, obj, params: Dict = {}, fit_method: str = 'fit'):
        self.fit_dict = {
            'method_name': fit_method,
            'params': params
        }
        return getattr(obj, fit_method)(x, **params)

    def track_decision_function(self, x ,obj, params: Dict = {},
        decision_function_method: str = 'decision_function'):
        self.decision_function_dict = {
            'method_name': decision_function_method,
            'params': params
        }
        return getattr(obj, decision_function_method)(x, **params)

    
    def store_recipe(self, yaml_path):
            yaml_content = {
                'init': self.init_dict,
                'fit': self.fit_dict,
                'decision_function': self.decision_function_dict
            }
            with open(yaml_path, "w+") as stream:
                 yaml.dump(yaml_content, stream)

We also define the helper function for performing sampling depending on choice of the user 

In [ ]:
#ID 25(5)
def sample_using_recipe(dataset_name,sampling_type,sampling_params):    
    
    if sampling_type == 'semisupervised_multiple':
      return benchmarking_datasets[dataset_name].sample_multiple(**sampling_params,yamlpath_append=f"{new_recipe_path}_{dataset_name}_preprocessing.yaml")
    elif sampling_type == 'semisupervised_explicit_numbers_single':
      return benchmarking_datasets[dataset_name].sample_with_explicit_numbers(**sampling_params,yamlpath_append=f"{new_recipe_path}_{dataset_name}_preprocessing.yaml")  
    elif sampling_type == 'semisupervised_training_split_multiple':
      return benchmarking_datasets[dataset_name].sample_multiple_with_training_split(**sampling_params,yamlpath_append=f"{new_recipe_path}_{dataset_name}_preprocessing.yaml")  
    elif sampling_type == 'semisupervised_training_split_single':        
      return benchmarking_datasets[dataset_name].sample_with_training_split(**sampling_params,yamlpath_append=f"{new_recipe_path}_{dataset_name}_preprocessing.yaml")   
    else:
        raise NotImplementedError(f"Sampling from yaml with type {type} is not implemented.")      



In [ ]:
#ID 26(5)
#-----------ALL ALGOS AND DATASETS FOR BENCHMARK RUN---------#


print("Datasets obtained from recipe:")    
for dataset in recipe_files_datasets:
    print(dataset_name)
#print(f"recipe_datasets:{recipe_datasets}")
#print(f"benchmarking_datasets: {benchmarking_datasets}")      

#print("\n") 
#print("Algos obtained from recipe:")    
for algo_name in recipe_files_algos:
    print(algo_name)
#print("\n")    
#print("ALl Datasets for this benchmark run:")    
for dataset_name in benchmarking_datasets:
    print(dataset_name)
#print(f"recipe_datasets:{    
#print("\n") 
#print("All algos for this benchmark run:")
for algo in lst_benchmark_algorithms:
    print(algo['algo_module_name'])


In [ ]:
#ID 27(5)                # RUNNING THE BENCHMARK
co = ComparisonObject()
for dataset_name in benchmarking_datasets:
    print(f'-------{dataset_name}-------') 
    
    #print(mvtec_ad_own_datasets_list)
    for alg in lst_benchmark_algorithms:
        name=alg["algo_module_name"]
        print(f"------{name}")
        eval_obj = EvaluationObject(algorithm_name=alg["algo_name_in_result_table"])
        algo_parameters=alg['algo_parameters']
        algo= getattr(__import__(alg["algo_module_name"]),alg["algo_class_name"]) # Algo object imported from class 
        algo_initialized=algo(**algo_parameters)
        seed=42
        torch.manual_seed(seed)
        random.seed(seed)
        
        
        tf.random.set_seed(seed)
        np.random.seed(seed) 
        os.environ['PYTHONHASHSEED'] = str(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        torch.use_deterministic_algorithms(True)
        if dataset_name[:9]=='mvtec_ad_' :  # if dataset is an 'mv_tec' dataset
            (x_train, x_test, y_test), sample_config in benchmarking_datasets[dataset_name].sample_original_mvtec_split(flatten_images=False)
            w = SemisupervisedWrapperToRecipe()
            w.track_init(algo, params=algo_parameters)
            w.track_fit(x=x_train, obj=algo_initialized, params=alg['fit']['params'], fit_method=alg['fit']['method_name']) # the last parameter is the name of the method used for fitting
            pred = w.track_decision_function(x_test,algo_initialized,params=alg['decision_function']['params'], decision_function_method=alg['decision_function']['method_name']) # the last parameter is the field name used to store anomaly scores by the model
            w.store_recipe(f"{new_recipe_path}_{name}_recipe.yaml")
            eval_obj.add(ground_truth=y_test, prediction=pred, description=sample_config)  
                    
            
        else:
    
         if dataset_name in recipe_files_datasets:
          for (x_train, x_test, y_test), sample_config in benchmarking_datasets[dataset_name].sample_from_yaml(recipe_files_datasets[dataset_name],type=sampling_type):
            print('.', end='') # update to see progress 
            w = SemisupervisedWrapperToRecipe()
            w.track_init(algo, params=algo_parameters)
            w.track_fit(x=x_train, obj=algo_initialized, params=alg['fit']['params'], fit_method=alg['fit']['method_name']) # the last parameter is the name of the method used for fitting
            pred = w.track_decision_function(x_test,algo_initialized,params=alg['decision_function']['params'], decision_function_method=alg['decision_function']['method_name']) # the last parameter is the field name used to store anomaly scores by the model
            w.store_recipe(f"{new_recipe_path}_{name}_recipe.yaml")
            eval_obj.add(ground_truth=y_test, prediction=pred, description=sample_config)
        
         else:
          
          for (x_train, x_test, y_test), sample_config in benchmarking_datasets[dataset_name].sample_using_recipe(dataset_name,sampling_type,sampling_params):
            print('.', end='') # update to see progress 
            w = SemisupervisedWrapperToRecipe()
            w.track_init(algo, params=algo_parameters)
            w.track_fit(x=x_train, obj=algo_initialized, params=alg['fit']['params'], fit_method=alg['fit']['method_name']) # the last parameter is the name of the method used for fitting
            pred = w.track_decision_function(x_test,algo_initialized,params=alg['decision_function']['params'] , decision_function_method=alg['decision_function']['method_name']) # the last parameter is the field name used to store anomaly scores by the model
            w.store_recipe(f"{new_recipe_path}_{name}_recipe.yaml")
            _append_to_yaml(f"{new_recipe_path}_{name}_recipe.yaml",'seed',seed)
            eval_obj.add(ground_truth=y_test, prediction=pred, description=sample_config)  
               
            
            
        eval_desc = eval_obj.evaluate(print=True,metrics=['roc_auc', 'adjusted_average_precision', 'precision_recall_auc'])
        co.add_evaluation(eval_desc)
        print("\n")
      

-------myImageDataset-------
------cae_iforest
.Evaluation on dataset mvtec_ad_myImageDataset with normal labels [0] and anomaly labels [1.0].
Total of 1 datasets. Per dataset:
89 training instances, 57 test instances, training contamination rate 0.0, test contamination rate 0.3157894736842105.
Mean 	 Std_dev 	 Metric
0.786 	 0.000 		 roc_auc
0.540 	 0.000 		 adjusted_average_precision
0.675 	 0.000 		 precision_recall_auc


------conv_ae
.Evaluation on dataset mvtec_ad_myImageDataset with normal labels [0] and anomaly labels [1.0].
Total of 1 datasets. Per dataset:
89 training instances, 57 test instances, training contamination rate 0.0, test contamination rate 0.3157894736842105.
Mean 	 Std_dev 	 Metric
0.786 	 0.000 		 roc_auc
0.481 	 0.000 		 adjusted_average_precision
0.634 	 0.000 		 precision_recall_auc


-------mnist-------
------cae_iforest
.Evaluation on dataset mnist with normal labels [0] and anomaly labels [1, 2, 3, 4, 5, 6, 7, 8, 9].
Total of 1 datasets. Per dataset:
483

# **6. EVALUATION**

Here , we will see how different metrics can be selected when evaluating an algorithm's performance.

In previous section while creating an evalutation description,  we used all metrics for evaluation:

     eval_desc = eval_obj.evaluate(print=False, metrics=all_metrics)
    
    

In [ ]:
#ID 28(6)

# to use a subset, first see which ones are available

print(all_metrics)

['roc_auc', 'average_precision', 'adjusted_average_precision', 'precision_n', 'adjusted_precision_n', 'precision_recall_auc']


In [ ]:
#ID 29(6)

#### ADD YOUR OWN NUMBER OF METRICS ###

#Then we can  select an arbitrary subset
metrics=['roc_auc', 'precision_recall_auc']

# **7. SHOW BENCHMARK RESULTS**

We compare by printing, the results of the evaluations of different Algo-Dataset combinations.

\[Latex version: bold for highest, italics for second highest, ?\]

In [ ]:
#ID 30(7)

# print results in easily readable format
co.print_results()

For roc_auc:
         mvtec_ad_myImageDataset     mnist   Average
CAE v2                  0.786325  0.982190  0.884257
conv_ae                 0.786325  0.935276  0.860801
Average                 0.786325  0.958733       NaN
For adjusted_average_precision:
         mvtec_ad_myImageDataset     mnist   Average
CAE v2                  0.540460  0.960543  0.750501
conv_ae                 0.480902  0.832317  0.656609
Average                 0.510681  0.896430       NaN
For precision_recall_auc:
         mvtec_ad_myImageDataset     mnist   Average
CAE v2                  0.675378  0.980264  0.827821
conv_ae                 0.633678  0.916091  0.774884
Average                 0.654528  0.948177       NaN


In [ ]:
#ID 31(7)

# print results in easily readable format with standard deviations
co.print_results(include_stdevs=True)

For roc_auc:
        mvtec_ad_myImageDataset         mnist   Average
CAE v2             0.786+-0.000  0.982+-0.000  0.884257
conv_ae            0.786+-0.000  0.935+-0.000  0.860801
Average                   0.786         0.959       NaN

For adjusted_average_precision:
        mvtec_ad_myImageDataset         mnist   Average
CAE v2             0.540+-0.000  0.961+-0.000  0.750501
conv_ae            0.481+-0.000  0.832+-0.000  0.656609
Average                   0.511         0.896       NaN

For precision_recall_auc:
        mvtec_ad_myImageDataset         mnist   Average
CAE v2             0.675+-0.000  0.980+-0.000  0.827821
conv_ae            0.634+-0.000  0.916+-0.000  0.774884
Average                   0.655         0.948       NaN



In [ ]:
#ID 32(7)

co.print_latex(include_stdevs=True)

For roc_auc:
\begin{center}
\begin{tabular}{  c c c c  }
  & mvtec\_ad\_myImageDataset & mnist & Average \\
  CAE v2 & \textbf{0.786$\pm$0.000} & \textbf{0.982$\pm$0.000} & \textbf{0.884} \\
  conv\_ae & \textit{0.786$\pm$0.000} & \textit{0.935$\pm$0.000} & \textit{0.861} \\
  Average & 0.786 & 0.959 &    \\
\end{tabular}
\end{center}

For adjusted_average_precision:
\begin{center}
\begin{tabular}{  c c c c  }
  & mvtec\_ad\_myImageDataset & mnist & Average \\
  CAE v2 & \textbf{0.540$\pm$0.000} & \textbf{0.961$\pm$0.000} & \textbf{0.751} \\
  conv\_ae & \textit{0.481$\pm$0.000} & \textit{0.832$\pm$0.000} & \textit{0.657} \\
  Average & 0.511 & 0.896 &    \\
\end{tabular}
\end{center}

For precision_recall_auc:
\begin{center}
\begin{tabular}{  c c c c  }
  & mvtec\_ad\_myImageDataset & mnist & Average \\
  CAE v2 & \textbf{0.675$\pm$0.000} & \textbf{0.980$\pm$0.000} & \textbf{0.828} \\
  conv\_ae & \textit{0.634$\pm$0.000} & \textit{0.916$\pm$0.000} & \textit{0.775} \\
  Average & 0.65

# **8. REPRODUCIBILITY**

 ## **8.1 Creating recipes**

This section shows **how `oab` can be used to make sampling results easily reproducible** .
 

`yaml` files play an integral role in making reproducibility work, as they store the operations and parameters performed on the data.

We will see how to produce a recipe(.yaml) of the Benchmarkrun already performed  in <b>#ID 24(5)</b>

In <b>#ID 11(3) #ID 13(3) and #ID 22(5) </b>,  We already performed `creating_recipe` operations on </b>own datasets and OAB's datasets as input recipe's datasets,we can see below  the structure of those recipes:

In [ ]:
#ID 33(8)

for (i,dataset_name) in enumerate(benchmarking_datasets):
    print(f"{i+1}.{new_recipe_path}_{dataset_name}_preprocessing.yaml\n")
    !cat {new_recipe_path}_{dataset_name}_preprocessing.yaml

1.Paper_A/20211207020855/20211207020855_Paper_A_cifar10_preprocessing.yaml

standard_functions:
- name: scale
  parameters:
    scaling_factor: 0.00392156862745098
anomaly_dataset:
  arguments:
    normal_labels: 0
    anomaly_labels:
sampling:
  semisupervised_training_split_multiple:
    training_split: 0.7
    max_contamination_rate: 0.5
    n_steps: 1
    random_seed: 42
    apply_random_seed: true
    keep_frequency_ratio_normals: false
    equal_frequency_normals: false
    keep_frequency_ratio_anomalies: false
    equal_frequency_anomalies: false
    flatten_images: false


Also, the algorithms which were run in <b>#ID 27(5)</b> whether from OAB , user as well as from input recipe are also stored in  new recipes, their recipes have been created in the same step of #ID 27(5), as we can see below: 

In [ ]:
#ID 34(8)
for (i,alg) in enumerate(lst_benchmark_algorithms):
        name=alg["algo_module_name"]
        print(f"{i+1}.{new_recipe_path}_{name}_recipe.yaml\n")
        !cat {new_recipe_path}_{name}_recipe.yaml

1.Paper_A/20211207020855/20211207020855_Paper_A_cae_iforest_recipe.yaml

init:
  params:
    CAE_parameters:
      latent_dim: 100
      epochs: 50
      verbose: 0
    IForest_parameters:
      random_state: 42
fit:
  method_name: fit
  params: {}
decision_function:
  method_name: decision_function
  params: {}
seed: 42
2.Paper_A/20211207020855/20211207020855_Paper_A_conv_ae_recipe.yaml

init:
  params:
    epochs: 50
    latent_dim: 100
    verbose: 0
fit:
  method_name: fit
  params: {}
decision_function:
  method_name: decision_function
  params: {}
seed: 42


Now, we will store the information of  datasets and algorithms information from <b>Paper_B's</b> recipe
and only of the algorithms of this benchmark in the new recipe
(execute this cell only  if Paper_B's recipe was used for benchmarking).

### 2. Reproducing the experiment

To reproduce the recipe created in the previous section,
we refer to <b>Section 5 #ID 22(5)</b> where we can reproduce the run as well as extend benchmarks!

# **9. EXTEND EXISTING BENCHMARK(own algorithm)**

To extend the existing benchmark here basically means to add  our own algorithm to the benchmark and to show the comparison results of pre-installed algorithms while also loading our own dataset.


1. We load the datasets. To know how to do that, we can refer to  **Section "1. Data" and "2. Data Selection"**
2. Then, load own algorithm as we will see in the next sub-section.

## **9.1 Loading own Algorithm**

In this subsection 5.1, you will see **how an own semisupervised anomaly detection algorithm** can easily be used within oab to be evaluated. We will see how a class representing an algorithm can be structured and how its performance is evaluated.

Of course, this is not the only way to use the functionality provided by oab. We do consider it to be the simplest way however.

In [ ]:
#ID 35(9)
# used imports from #ID 2,#ID 3,#ID 8,#ID 18,#ID 21 for ths whole section
# download example algorithm and inspect content
import wget
wget.download('https://raw.githubusercontent.com/jandeller/test/main/RandomGuesserSemisupervised.py', "RandomGuesserSemisupervised.py")
!cat RandomGuesserSemisupervised.py

import numpy as np

class RandomGuesserSemisupervised():

    def fit(self, X_train):
        pass
      
    def decision_function(self, X_test):
        "Assign a random number to each sample from the test set"
        n_samples = X_test.shape[0]
        return np.random.randn(n_samples)


The sample `RandomGuesser` algorithm shown here is - as the name suggests - a random guesser, i.e., it assigns random anomaly scores to the samples.

An algorithm used for semisupervised anomaly detection needs to specify a `fit(X_train)` method for training and a `decision_function(X_test)` method for inference that returns an anomaly score per data point in the test set.

It is of course possible to rename the method and field, use a method for accessing the anomaly scores, etc. Note that if this is done, the following code has to be changed accordingly. Adhering to the conventions described above (`fit(X_train)` and `decision_function(X_test)`) allows you to use the same interface as algorithms from [`PyOD`](https://pyod.readthedocs.io/en/latest/) as shown when [comparing algorithms using `oab`](https://colab.research.google.com/drive/1aV_itaYCJgzdZ1lQ7SUyHQ7z01xSPxDN?usp=sharing#scrollTo=QnAfCGTGL7xv).

In [ ]:
#ID 36(9)
# used imports from #ID 2(1),#ID 18(5)
#used sampling parameters from #ID 15(4)
#used benchmarking_datasets from #ID 16(4)

# and import the RandomGuesser
from RandomGuesserSemisupervised import RandomGuesserSemisupervised

# looking at all the benchmarking datasets again
for dataset_name in benchmarking_datasets:
    print(dataset_name)
    #print(benchmarking_datasets[dataset_name])



myImageDataset
mnist


In [ ]:
#ID 37(9)

#  A comparison object is created for comparing the evaluations of different Algo-Dataset combinations
co1 = ComparisonObject()

for dataset in benchmarking_datasets:
  # evaluate the random guesser
  eval_obj1 = EvaluationObject(algorithm_name="RandomGuesser")
  for (X_train, X_test, y_test), settings in benchmarking_datasets[dataset_name].sample_multiple_with_training_split(training_split=training_split, 
                                                                  max_contamination_rate=max_contamination_rate, 
                                                                  n_steps=n_steps):
      print(".", end=" ") # update to see progress
      rg = RandomGuesserSemisupervised()
      rg.fit(X_train) # data is fitted to RandomGuesser
      pred = rg.decision_function(X_test) # and decision_scores_ is accessed
      eval_obj1.add(y_test, pred, settings)

  eval_desc = eval_obj1.evaluate()
  # added to comparison object
  co1.add_evaluation(eval_desc)
  print("\n")



. Evaluation on dataset mnist with normal labels [0] and anomaly labels [1, 2, 3, 4, 5, 6, 7, 8, 9].
Total of 1 datasets. Per dataset:
4832 training instances, 4142 test instances, training contamination rate 0.0, test contamination rate 0.5.
Mean 	 Std_dev 	 Metric
0.501 	 0.000 		 roc_auc
0.503 	 0.000 		 average_precision
0.006 	 0.000 		 adjusted_average_precision


. Evaluation on dataset mnist with normal labels [0] and anomaly labels [1, 2, 3, 4, 5, 6, 7, 8, 9].
Total of 1 datasets. Per dataset:
4832 training instances, 4142 test instances, training contamination rate 0.0, test contamination rate 0.5.
Mean 	 Std_dev 	 Metric
0.501 	 0.000 		 roc_auc
0.503 	 0.000 		 average_precision
0.006 	 0.000 		 adjusted_average_precision




As in the above code, We store the evaluations of our own algorithm in evaluation object which is then added to comparison object.Similarly, we can create evaluation objects for other algorithms and add them to comparison object for final benchmarking  as shown in Section 5

Finally, we show below the benchmarking results of our algorithm as described in "**Section 7. Show Benchmarking Results**"

In [ ]:
#ID 38(9)

# print results in easily readable format
co1.print_results()

For roc_auc:
                  mnist   Average
RandomGuesser  0.500874  0.500874
Average        0.500874       NaN
For average_precision:
                  mnist   Average
RandomGuesser  0.503196  0.503196
Average        0.503196       NaN
For adjusted_average_precision:
                  mnist   Average
RandomGuesser  0.006392  0.006392
Average        0.006392       NaN


In [ ]:
#ID 39(9)
# print results in easily readable format with standard deviations
co1.print_results(include_stdevs=True)

For roc_auc:
                      mnist   Average
RandomGuesser  0.501+-0.000  0.500874
Average               0.501       NaN

For average_precision:
                      mnist   Average
RandomGuesser  0.503+-0.000  0.503196
Average               0.503       NaN

For adjusted_average_precision:
                      mnist   Average
RandomGuesser  0.006+-0.000  0.006392
Average               0.006       NaN



So,This was our example algorithm. Other algorithms can be used to run and extend benchmarks,  Please refer  to <b>#ID 19(5)</b>.